In [1]:
!pip install git+https://github.com/lnccbrown/hssm@main
!pip install git+https://github.com/brown-ccv/hddm-wfpt@main
!pip install numpyro

zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


In [1]:
!pip install dask

You should consider upgrading via the '/Users/jonner/.pyenv/versions/3.10.2/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import os
import sys
import time
from matplotlib import pyplot as plt
import arviz as az
import pytensor
import hssm
import hddm_wfpt
import bambi as bmb
from ssms.basic_simulators import simulator
import pandas as pd
import numpy as np
import dask
import arviz as az
import dask

from arviz.utils import conditional_jit, Dask

In [3]:
hssm.set_floatX("float64")

Setting PyTensor floatX type to float64.
Setting "jax_enable_x64" to True. If this is not intended, please set `jax` to False.


In [5]:
data=pd.read_csv('ddm.txt', sep='\t')
data = data[data['response'] !=4]

In [15]:
#v ~ 1 + stim:C(cond)
m = hssm.HSSM(data=data[['rt', 'response', 'Type1','Type2']], include=[
        {
            "name": "v",
            "formula": "v ~ 1 + (1|Type1) + (1|Type2)",
            "link": "identity",
        },
        ], model='ddm')

In [ ]:
infer_data_simple_ddm_model = m.sample(
    sampler="nuts_numpyro",  # type of sampler to choose, 'nuts_numpyro', 'nuts_blackjax' of default pymc nuts sampler
    cores=5,  # how many cores to use
    chains=5,  # how many chains to run
    draws=120,  # number of draws from the markov chain
    tune=120,  # number of burn-in samples
    idata_kwargs=dict(log_likelihood=True),  # return log likelihood
)  # mp_ctx="forkserver")

In [ ]:
infer_data_simple_ddm_model

In [ ]:
Dask.enable_dask(dask_kwargs={"dask": "parallelized", "output_dtypes": [float]})

In [ ]:
az.plot_trace(
    infer_data_simple_ddm_model,
    var_names="~log_likelihood",  # we exclude the log_likelihood traces here
    legend = True
)
plt.tight_layout()

In [15]:
az.summary(infer_data_simple_ddm_model)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
v,-3.194,0.395,-4.022,-2.597,0.050,0.035,64.0,67.0,1.04
a,2.184,0.690,1.178,3.606,0.081,0.057,69.0,92.0,1.06
t,0.261,0.108,0.021,0.397,0.014,0.010,63.0,62.0,1.03
z,0.546,0.140,0.299,0.776,0.015,0.010,93.0,102.0,1.05
